## Data Cleaning

* Data cleaning is data set specific but there are some common problems
    * Missing values
    * Duplicates
    * Reporting or collection bias or drift
    * Reporting or collection errors
* Data cleaning is more effective when it is combined with visualization and some preliminary modeling
* Decribe all the steps you take to load and clean the data so that you and others can repeat the process

### In Class Exercise

Use your functions or the ones below to load the train data into a dataframe and combine all the narrative fields into one variable: Narrative. Optionally, remove the previous narrative columns.

In [42]:
# Packages for loading, cleaning, visualization, and analysis

# Data
import pandas as pd
import numpy as np
import scipy as sp
import os
import string as st


#### I decided to use my own functions that I had made

In [85]:
# I have my own function that will pull each year of the train data in a panda df
# You pass a directory and it returns a list of pandas dataframes
def getListDataFrames(path):
    # initialize an empty list
    csvs = list()
    # Get all the files in the directory
    all_files = os.listdir(path)
    # Iterate across each of the files in the Data folder
    for fileName in all_files:
        # Get the full path name
        fileToPull = path + fileName
        # Read in the data at that location
        filesOfInterest = pd.read_csv(fileToPull, index_col = False, low_memory = False)
        csvs.append(filesOfInterest)
    return csvs

# Here is the list of 16 pandas dataframes corresponding to 2001 thru 2016
listOfAccidents = getListDataFrames('/Users/mead/Fall2017/DonBrown-DS6001/InClass1/Data/')

#### Concatenate the pandas together

In [93]:
acc_df = pd.concat(listOfAccidents, ignore_index = True)
acc_df.shape

(51623, 153)

#### Combine the Narratives

In [94]:
# 1
# Produce a function to add all of the Narratives together
# Takes the index of the NARR1 column as input alongside the dataframe of interest. 
# It outputs the full Narrative column
def combineNARR(NARR1index, dataFrme):
# Initialize an empty list of all narratives
    allAccs = list()
    # Iterate across every row
    for acc in range(len(dataFrme.index)):
        # Initialize an empty list of each row's narrative
        fullNarr = list()
        # Iterate across every NARR column
        for narr in range(15):
            index = narr + NARR1index[0]
            # Keep track of the value in that NARR column for that row
            narrElement = str(dataFrme.iloc[acc, index])
            fullNarr.append(narrElement)
        allAccs.append(fullNarr)
    return allAccs

# This is the which function which keeps track of where NARR1 is
which = lambda lst:list(np.where(lst)[0])
NARR1index = which([name == 'NARR1' for name in acc_df.columns])

# Run the function -- output is a list of lists
newColumn = combineNARR(NARR1index, acc_df)

# 2
# Combine all the NARR together for each row. Make sure to remove the NAs
Narratives = [''.join([sentence for sentence in NARR if sentence != 'nan']) for NARR in newColumn]
# Add this as a new column for the accidents df
acc_df['Narrative'] = pd.Series(Narratives, index=acc_df.index)

#### Drop old NARR columns

In [95]:
# Drop old narrative columns
narrList = []
for i in range(0,15):
    a = str(i + 1)
    narrList.append('NARR' + a)
acc_df.drop(narrList, axis = 1, inplace = True)

## Duplicates

- Why should we remove duplicates?
    - Remove duplicates because a duplicate is going to be regarded as more influential than a single point
- How should we remove them?
    - Search for columns with all duplicate values (unique/duplicate function or something like that)

In [97]:
# Looking for duplicates
# I know that there is something specific in the readme about which columns are
# duplicates but I just used the narrative since the seems to be good enough
print(len(acc_df.loc[acc_df[['Narrative']].duplicated()]))

5372


Looks like there are around 5,372 duplicated values.

In [140]:
# Removing the duplicates
acc_clean_df = acc_df.drop_duplicates('Narrative')

## Missing Values

There are essentially 3 ways to handle missing values:

    1. Remove the columns (variables) with missing values
    2. Remove the row (observations) with missing values
    3. Impute the missing values

The choice of which of these to use depends on the problem and the data. If a variable does not seem important to the problem or if it has many missing values, then eliminating it is reasonable. Similarly if the observation appears to not represent the data or if it has many missing values, then eliminating it seems reasonable.
Imputation

Imputation can be done in may ways. The most common are the following:

    1. Replace the missing value with the mean
    2. Replace the missing value with the median
    3. Replace the missing value with the mode
    4. Use k-nn to
        1. Replace the missing value with the mean
        2. Replace the missing value with the median
        3. Replace the missing value with the mode
 
Options 1 and 2 can only be used with numeric or quantitative variables.

### In Class Exercise

1. Find out how many missing values we have by column (variable).
2. What technique or techniques should we use to handle them?
3. What methods from pandas or sklearn can we use for imputation? Why or how would we use them?

### In Class Exercise Answers

1. Many of the columns have a large proportion of values that are NAs. By column is calculated below. Across all columns there are 1,788,460 missing values
2. We talk above about this above, but the general idea is that for missing values we can either:
    1. Remove the column if most of the values are missing
    2. Remove the row if it's missing fundamentally important info
    3. Use the mean or median for quantitative variables (imputation)
    4. Use the mode for categorical variables. (imputation)
3. Techniques for imputation
    1. Pandas
        1. Provides isnull() to specifically return missing values. Has a large numberof uses.
        2. Can use fillna(value) to replace all the na's with a particular value. Useful if you know that missing values have some significance in the original dataset
        3. fillna(method='pad') let's you use the most recent non-missing value. This is useful for time series when you want to consider the most recently-available value
        4. Can also use fillna(acc_df.mean()) to fill the missing values in each column with the mean value. Applications to quantitative variables are immediate. (also use with median and mode)
        5. dropna() lets you get entirely rid of columns (or rows) with missing data. Greatif you decide you want no missing data at all.
        6. There is an interpolate() method that allows you to do linear interpolation for missing data. This would be good for time series data.
    2. sklearn
        1. sklearn has an Imputer class which you can initialize and use with a number of different imputation strategies (eg: mean, median, and mode), to replace missing values.
        2. You can also build your own imputer class (example below), inheriting from sklearn's TransformerMixin class, which gives you even more control in transforming your data when you run into missing values. In the example below the method was implemented to look at the datatypes of missing values in order to decide whether to do mean, median, or mode.

In [100]:
# 1
print("As a reminder, the size of our panda is: " + str(acc_clean_df.shape))
print("Here are the missing values by column: \n" + str(acc_clean_df.isnull().sum()))
print("And that's a total of " + str(acc_clean_df.isnull().sum().sum()) + " missing values across all columns")

As a reminder, the size of our panda is: (46251, 139)
Here are the missing values by column: 
ACCAUSE          0
ACCDMG           0
ACCTRK         113
ACCTRKCL       134
ADJUNCT1     38918
ADJUNCT2     45446
ADJUNCT3     45962
ALCOHOL      33633
AMPM             7
AMTRAK       43336
BRAKEMEN      4113
CABOOSE1         0
CABOOSE2         0
CARNBR1       3417
CARNBR2      41302
CARS             0
CARSDMG          0
CARSHZD          0
CASINJ           0
CASINJRR         0
CASKLD           0
CASKLDRR         0
CAUSE            0
CAUSE2       40558
CDTRHR       11636
CDTRMIN      16061
CNTYCD          14
CONDUCTR      2385
COUNTY           0
CauseCat     46251
             ...  
SUBDIV       34256
TEMP             0
TIMEHR           0
TIMEMIN          0
TONS             0
TOTINJ           0
TOTKLD           0
TRKCLAS        465
TRKDMG           0
TRKDNSTY     25338
TRKNAME        483
TRNDIR        2911
TRNNBR        6373
TRNSPD           0
TYPE             0
TYPEQ         3404
TYPRR        

In [101]:
# This class imputes the missing values as
# (1) most frequent if the variable is categorical 
# (2) mean if the variable is real (floating point)
# (3) median if the variable is an integer 

# Here is a class that will provide imputation
# This is an extension by D.Brown to sveitser, 2014 https://stackoverflow.com/users/469992/sveitser

from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.
        
        Columns of dtype floating point are imputed with the mean.

        Columns of other types are imputed with median of the column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') 
                               else X[c].mean() if X[c].dtype == np.dtype('f')
                                else X[c].median() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)


## Categorical Variables

Categorical variables must be treated differently from quantitative variables. Their effects on the modeling
and machine learning depend on correctly coding them for the analysis. 

Categorical variables can be entered in many diffent ways. Data scientists should inspect the coding 
of these variables to insure correctness, approrpriateness for modeling, and easy interpretation.

### In Class Exercise

1. Look at the data types for the variables.
2. Which variables are categorical? Look at some of their value_counts()
3. Which variables are categorical but are coded as integers? Which variables are integers but coded as objects?
4. Replace integer values for TYPE with the text labels. Repeat for one other variable.

### In Class Exercise Answers

1. Shown below. Easy to access.
2. I used http://safetydata.fra.dot.gov/OfficeofSafety/publicsite/Newregulation.aspx?doc=accfile_EFFECTIVE_060111.pdf to decide these. So the categorical variables are amtrak, iyr, imo, railroad, iyr2, imo2, rr2, iyr3, imo3, rr3, gxid, year, month, ampm, type, division, station, milepost, state, visibility, weather, typspd, trnnbr, trndir, typeq, equatt, trkname, trkclass, typtrk, rrcar1, carnbr1, loaded1, rrcar2, carnbr2, loaded2, cause, cause2, acccause, acctrk, acctrkcl, stcnty, jointcd, region, typrr, rrdiv, method, year4, county, cntycd, passtrn, ssb1, ssb2, Narrative, rcl, signal, mopera, adjunct1, adjunct2, sdjunct3, subdiv  
3. Miscoding (look below for code):
    1. Variables coded as integers but are actually categorical:
        1. IMO, IMO3, IYR, IYR3, JOINTCD, MONTH, REGION, STATE, TYPE, VISIBLTY, WEATHER, YEAR, YEAR4
    2. Variables coded as objects but are actually integers:
        1. BRAKEMEN, CONDUCTR, ENGRS, FIREMEN, INCDTNO, INCDTNO2, INCDTNO3
4. Done below for TYPE and for WEATHER

In [135]:
# 1
# Just getting data types. Easy
acc_df.dtypes

ACCAUSE       object
ACCDMG       float64
ACCTRK       float64
ACCTRKCL      object
ADJUNCT1      object
ADJUNCT2      object
ADJUNCT3      object
ALCOHOL      float64
AMPM          object
AMTRAK        object
BRAKEMEN      object
CABOOSE1     float64
CABOOSE2     float64
CARNBR1       object
CARNBR2      float64
CARS         float64
CARSDMG      float64
CARSHZD      float64
CASINJ       float64
CASINJRR     float64
CASKLD       float64
CASKLDRR     float64
CAUSE         object
CAUSE2        object
CDTRHR       float64
CDTRMIN      float64
CNTYCD       float64
CONDUCTR      object
COUNTY        object
CauseCat     float64
              ...   
SUBDIV        object
TEMP         float64
TIMEHR       float64
TIMEMIN      float64
TONS         float64
TOTINJ       float64
TOTKLD       float64
TRKCLAS       object
TRKDMG       float64
TRKDNSTY      object
TRKNAME       object
TRNDIR       float64
TRNNBR        object
TRNSPD         int64
TYPE           int64
TYPEQ         object
TYPRR        

In [136]:
# 2
#print("There are " + str(len(acc_df.columns[acc_df.dtypes == 'object'])) + " variables which are 'object' (so considered categorical.)")
#print("These are the variables considered as categorical 'objects': " + str(acc_df.columns[acc_df.dtypes == 'object']) + "\n")
print("And these are all of their value_counts(): \n")
# Code to look at freq of value_counts for each 
cat_list = acc_clean_df[['YEAR', 'AMPM', 'DIVISION', 'TRNDIR', 'TYPE', 'TYPEQ', 'TYPTRK', 'VISIBLTY', 'WEATHER']].columns#[acc_clean_df.dtypes == 'object']
for i in cat_list:
    print(str(i) + "\n" + str(acc_clean_df[i].value_counts()) + "\n")

And these are all of their value_counts(): 

YEAR
4     4018
5     3885
1     3757
3     3646
6     3637
2     3388
7     3332
8     2904
11    2414
10    2335
9     2317
15    2256
13    2192
14    2112
12    2084
16    1974
Name: YEAR, dtype: int64

AMPM
PM    23903
AM    22341
Name: AMPM, dtype: int64

DIVISION
SYSTEM                  4398
CHICAGO                 1419
GULF                     812
TWIN CITIES              771
SPRINGFIELD              744
NORTH LITTLE ROCK        664
HOUSTON                  633
KANSAS                   610
TEXAS                    595
NORTHWEST                555
PORTLAND                 508
ROSEVILLE                495
ATLANTA                  488
POWDER RIVER             466
ALBANY                   465
CENTRAL                  447
JACKSONVILLE             445
NEBRASKA                 434
NORTH PLATTE             426
SAN ANTONIO              408
FLORENCE                 408
FORT WORTH               404
DENVER                   382
SOUTHWEST        

In [137]:
# 3
print("These are the variables considered as integer : " + str(acc_df.columns[acc_df.dtypes == 'int']) + "\n")
print("These are the variables considered as 'objects': " + str(acc_df.columns[acc_df.dtypes == 'object']) + "\n")

These are the variables considered as integer : Index([u'DAY', u'DUMMY3', u'HIGHSPD', u'IMO', u'IMO3', u'IYR', u'IYR3',
       u'JOINTCD', u'MONTH', u'POSITON1', u'POSITON2', u'REGION', u'STATE',
       u'TRNSPD', u'TYPE', u'VISIBLTY', u'WEATHER', u'YEAR', u'YEAR4'],
      dtype='object')

These are the variables considered as 'objects': Index([u'ACCAUSE', u'ACCTRKCL', u'ADJUNCT1', u'ADJUNCT2', u'ADJUNCT3', u'AMPM',
       u'AMTRAK', u'BRAKEMEN', u'CARNBR1', u'CAUSE', u'CAUSE2', u'CONDUCTR',
       u'COUNTY', u'DIVISION', u'DUMMY4', u'ENGRS', u'EQATT', u'FIREMEN',
       u'GXID', u'INCDTNO', u'INCDTNO2', u'INCDTNO3', u'LOADED1', u'LOADED2',
       u'METHOD', u'MILEPOST', u'PASSTRN', u'RAILROAD', u'RR2', u'RR3',
       u'RRCAR1', u'RRCAR2', u'RRDIV', u'SSB1', u'SSB2', u'STATION', u'STCNTY',
       u'SUBDIV', u'TRKCLAS', u'TRKDNSTY', u'TRKNAME', u'TRNNBR', u'TYPEQ',
       u'TYPRR', u'TYPSPD', u'Narrative'],
      dtype='object')



In [141]:
# 4
# Now to actually go in and make replacements for all of these values 
# you can go in and use .replace()
acc_clean_df["TYPE"] = acc_clean_df['TYPE'].replace(range(1,14),['Derailment', 
    'Head on collision', 'Rearend collision', 'Side collision', 'Raking collision', 
    'Broken train collision', 'Hwy-rail crossing', 'RR Grade Crossing', 'Obstruction', 
    'Explosive-detonation', 'fire/violent rupture', 'Other impacts', 
    'Other (described in narr)'])
# Or you can go in and use a dictionary
#map_typeq = {1:'Freight', '1':'Freight'}
#acc_clean_df['TYPEQ'] = acc_clean_df.map(map_typeq)
print("Types: \n" + str(acc_clean_df["TYPE"].value_counts()))

# Doing the same thing  but for weather
acc_clean_df["WEATHER"] = acc_clean_df['WEATHER'].replace(range(1,7),['clear', 
    'cloudy', 'rain', 'fog', 'sleet', 'snow'])
print("\nWeather: \n" + str(acc_clean_df["WEATHER"].value_counts()))

Types: 
Derailment                   29902
Other impacts                 4861
Hwy-rail crossing             3786
Other (described in narr)     2343
Side collision                2071
Obstruction                   1196
Raking collision               856
fire/violent rupture           522
Rearend collision              409
Head on collision              191
Broken train collision          91
Explosive-detonation            15
RR Grade Crossing                8
Name: TYPE, dtype: int64

Weather: 
clear     30936
cloudy    10471
rain       3212
snow       1001
fog         536
sleet        95
Name: WEATHER, dtype: int64
